In [1]:
import numpy as np, matplotlib.pyplot as plt, pandas as pd
pd.set_option('display.max_rows', 40)

# Use this to see how much memory the dataframes use
from sys import getsizeof

from vivarium_research_prl.noise import corruption, fake_names, noisify
from vivarium_research_prl.find_kids import datasets, noisify_data

!date
!whoami
!uname -a
!pwd

Sat 04 Mar 2023 09:07:58 AM PST
ndbs
Linux int-slurm-sarchive-p0002 5.4.0-135-generic #152-Ubuntu SMP Wed Nov 23 20:19:22 UTC 2022 x86_64 x86_64 x86_64 GNU/Linux
/mnt/share/code/ndbs/vivarium_research_prl/linkage/wic_case_study/v2.0


In [2]:
%load_ext autoreload
%autoreload 2

# Find and load data

### Runs posted by Steve and Jim in Slack on Tue 2/28/2023

From Steve:
```
/mnt/team/simulation_science/priv/engineering/vivarium_census_prl_synth_pop/results/vv_business_moves_dynamics/united_states_of_america/2023_02_24_09_03_36/final_results/2023_02_27_17_03_13/
```

From Jim:
```
/mnt/team/simulation_science/priv/engineering/vivarium_census_prl_synth_pop/results/vv_mailing_addresses/united_states_of_america/2023_02_27_10_25_20/final_results/2023_02_27_14_59_44
```

Going to start with the second one.

- 788 MB on disk for compressed decennial census observer
- 74 MB on disk for compressed WIC observer

In [3]:
project_output_dir = \
    '/mnt/team/simulation_science/priv/engineering/vivarium_census_prl_synth_pop'
output_subdir = (
    'results/vv_mailing_addresses'
    '/united_states_of_america/2023_02_27_10_25_20'
    '/final_results/2023_02_27_14_59_44'
)
output_dir = f'{project_output_dir}/{output_subdir}'

!ls -halt $output_dir


total 7.4G
drwxrwsr-x 3 albrja IHME-Simulationscience 1.0K Feb 27 17:50 ..
-rw-r--r-- 1 albrja IHME-Simulationscience 6.4G Feb 27 17:49 tax_w2_observer.csv.bz2
drwxrwsr-x 2 albrja IHME-Simulationscience 3.0K Feb 27 16:46 .
-rw-r--r-- 1 albrja IHME-Simulationscience 141M Feb 27 16:37 social_security_observer.csv.bz2
-rw-r--r-- 1 albrja IHME-Simulationscience  71M Feb 27 16:35 wic_observer.csv.bz2
-rw-r--r-- 1 albrja IHME-Simulationscience  40M Feb 27 16:34 household_survey_observer_cps.csv.bz2
-rw-r--r-- 1 albrja IHME-Simulationscience 7.9M Feb 27 16:33 household_survey_observer_acs.csv.bz2
-rw-r--r-- 1 albrja IHME-Simulationscience 752M Feb 27 16:33 decennial_census_observer.csv.bz2


In [4]:
%%time
df_census_raw = pd.read_csv(f'{output_dir}/decennial_census_observer.csv.bz2')
df_census_raw

<timed exec>:1: DtypeWarning: Columns (26,30) have mixed types. Specify dtype option on import or set low_memory=False.


CPU times: user 3min 16s, sys: 12.3 s, total: 3min 28s
Wall time: 3min 28s


,simulant_id,first_name_id,middle_name_id,last_name_id,age,sex,race_ethnicity,date_of_birth,address_id,state_id,...,street_name,unit_number,city,mailing_address_street_number,mailing_address_street_name,mailing_address_unit_number,mailing_address_po_box,mailing_address_state,mailing_address_city,mailing_address_zipcode
0,3254_0,3254_0,3254_0,3254_0,59.550114,Male,White,1960-09-12,3254_6,25,...,cliffside dr,NaN,billerica,NaN,cliffside dr,NaN,0,MA,billerica,2740
1,3254_1,3254_1,3254_1,3254_0,58.674887,Female,White,1961-07-29,3254_6,25,...,cliffside dr,NaN,billerica,NaN,cliffside dr,NaN,0,MA,billerica,2740
2,3254_2,3254_2,3254_2,3254_2,53.272919,Male,White,1966-12-23,3254_7,55,...,108th ave se,NaN,milwaukee,14323,108th ave se,NaN,0,WI,milwaukee,53952
3,3254_3,3254_3,3254_3,3254_2,54.643021,Female,White,1965-08-10,3254_7,55,...,108th ave se,NaN,milwaukee,14323,108th ave se,NaN,0,WI,milwaukee,53952
4,3254_4,3254_4,3254_4,3254_4,55.642824,Female,Black,1964-08-09,3254_8,24,...,ne 7th ave,NaN,nrt beach,358,ne 7th ave,NaN,0,MD,nrt beach,21239
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15448912,3541_883506,3541_883506,3541_883506,3541_883504,7.739110,Male,White,2022-07-21,3541_618378,6,...,lone wolf dr,NaN,tulare,NaN,NaN,NaN,4657,CA,san diego,95136
15448913,3541_883507,3541_883507,3541_883507,3541_883504,31.564120,Female,White,1998-09-23,3541_618378,6,...,lone wolf dr,NaN,tulare,NaN,NaN,NaN,4657,CA,san diego,95136
15448914,3541_883508,3541_883508,3541_883508,3541_883508,27.369618,Male,Latino,2002-12-03,3541_618379,37,...,old silo rd,NaN,mooresville,1780,old silo rd,NaN,0,NC,mooresville,27217
15448915,3541_883509,3541_883509,3541_883509,3541_883509,71.134048,Male,Asian,1959-02-27,3541_618380,6,...,blue rdge ave,NaN,visalia,565,blue rdge ave,NaN,0,CA,visalia,91770


# Check memory usage and datatypes

In [5]:
%%time
getsizeof(df_census_raw) # 24 GB for 15M rows

CPU times: user 22.7 s, sys: 231 ms, total: 22.9 s
Wall time: 22.9 s


23755355459

In [6]:
df_census_raw.dtypes

simulant_id                       object
first_name_id                     object
middle_name_id                    object
last_name_id                      object
age                              float64
sex                               object
race_ethnicity                    object
date_of_birth                     object
address_id                        object
state_id                           int64
po_box                             int64
puma                               int64
guardian_1                         int64
guardian_2                         int64
guardian_1_address_id            float64
guardian_2_address_id            float64
relation_to_household_head        object
census_year                        int64
housing_type                      object
random_seed                        int64
year_of_birth                      int64
state                             object
first_name                        object
middle_initial                    object
last_name       

In [7]:
sim_id_size = getsizeof(df_census_raw['simulant_id']) # 1GB for simulant_id column (str)
print(sim_id_size / 1e6, sim_id_size / len(df_census_raw))

1048.430004 67.86430427453264


In [8]:
age_size = getsizeof(df_census_raw['age']) # 123 MB for age column (float64)
print(age_size / 1e6, age_size / len(df_census_raw))

123.59148 8.000009321041727


In [9]:
zipcode_size = getsizeof(df_census_raw['zipcode']) # 123 MB for zipcode column (int64)
print(zipcode_size / 1e6, zipcode_size / len(df_census_raw))

123.59148 8.000009321041727


# See why some columns were loaded with mixed types

It's street numbers that are either NaN or not all digits.

In [64]:
# These columns had mixed dtype, aparently
df_census_raw.iloc[:,[26,30]]

,street_number,mailing_address_street_number
0,NaN,NaN
1,NaN,NaN
2,14323,14323
3,14323,14323
4,358,358
...,...,...
15448912,3589,NaN
15448913,3589,NaN
15448914,1780,1780
15448915,565,565


In [71]:
street_num_types = df_census_raw['street_number'].map(type)
street_num_types.unique()

array([<class 'float'>, <class 'str'>], dtype=object)

In [74]:
df_census_raw.loc[street_num_types==float, 'street_number']

0           NaN
1           NaN
115         NaN
116         NaN
117         NaN
           ... 
15448781    NaN
15448782    NaN
15448783    NaN
15448800    NaN
15448801    NaN
Name: street_number, Length: 727534, dtype: object

In [78]:
street_num_floats = df_census_raw.loc[street_num_types==float, 'street_number']
street_num_floats.isna().all()

False

In [79]:
street_num_floats.loc[street_num_floats.notna()]

3817472     1127.0
3817473     1127.0
3817474      119.0
3817475      119.0
3817476      119.0
             ...  
13107195     949.0
13107196     949.0
13107197     898.0
13107198     949.0
13107199     747.0
Name: street_number, Length: 46743, dtype: object

In [73]:
df_census_raw.loc[street_num_types==str, 'street_number']

2           14323
3           14323
4             358
5             358
6             358
            ...  
15448912     3589
15448913     3589
15448914     1780
15448915      565
15448916    28014
Name: street_number, Length: 14721383, dtype: object

In [80]:
street_num_str = df_census_raw.loc[street_num_types==str, 'street_number']
street_num_str.str.isdigit().all()

False

In [82]:
street_num_str.loc[~street_num_str.str.isdigit()]

542         n88w 17907
543         n88w 17907
544         n88w 17907
545         n88w 17907
632             144-46
               ...    
15448247      5862 1 2
15448363         w3603
15448456        950 a9
15448615      1000 1 2
15448703        207-18
Name: street_number, Length: 183590, dtype: object

# Write a function to process census into final expected form

In [10]:
df_census_raw.census_year.unique()

array([2020, 2030])

In [11]:
df_census_raw.dtypes

simulant_id                       object
first_name_id                     object
middle_name_id                    object
last_name_id                      object
age                              float64
sex                               object
race_ethnicity                    object
date_of_birth                     object
address_id                        object
state_id                           int64
po_box                             int64
puma                               int64
guardian_1                         int64
guardian_2                         int64
guardian_1_address_id            float64
guardian_2_address_id            float64
relation_to_household_head        object
census_year                        int64
housing_type                      object
random_seed                        int64
year_of_birth                      int64
state                             object
first_name                        object
middle_initial                    object
last_name       

In [12]:
%%time

def get_single_census(df_census_raw, year):
    columns_for_census = [
        'simulant_id', 'relation_to_household_head',
        'date_of_birth', 'age', 'sex', 'race_ethnicity',
        'first_name', 'middle_initial', 'last_name',
        'street_number', 'street_name', 'unit_number',
        'city', 'state', 'zipcode',
    ]
    df_census = df_census_raw.loc[
        df_census_raw.census_year==year, columns_for_census]
    df_census['sim_id_int'] = df_census['simulant_id'].str.split('_')
    return df_census

df_census = get_single_census(df_census_raw, 2020)
df_census

CPU times: user 12.2 s, sys: 1.18 s, total: 13.4 s
Wall time: 13.4 s


,simulant_id,relation_to_household_head,date_of_birth,age,sex,race_ethnicity,first_name,middle_initial,last_name,street_number,street_name,unit_number,city,state,zipcode,sim_id_int
0,3254_0,Reference person,1960-09-12,59.550114,Male,White,David,T,Allen,NaN,cliffside dr,NaN,billerica,MA,2740,"[3254, 0]"
1,3254_1,Opp-sex spouse,1961-07-29,58.674887,Female,White,Deanna,S,Allen,NaN,cliffside dr,NaN,billerica,MA,2740,"[3254, 1]"
2,3254_2,Reference person,1966-12-23,53.272919,Male,White,David,T,Dudich,14323,108th ave se,NaN,milwaukee,WI,53952,"[3254, 2]"
3,3254_3,Opp-sex spouse,1965-08-10,54.643021,Female,White,Dawn,T,Dudich,14323,108th ave se,NaN,milwaukee,WI,53952,"[3254, 3]"
4,3254_4,Reference person,1964-08-09,55.642824,Female,Black,Deanna,T,Jones,358,ne 7th ave,NaN,nrt beach,MD,21239,"[3254, 4]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14654635,3541_751066,Reference person,1991-09-28,28.584974,Male,Asian,Michael,J,Sandhu,826,knight st,NaN,oswego,IL,60607,"[3541, 751066]"
14654636,3541_751067,Parent,1963-08-30,56.666066,Female,Asian,Carrie,J,Sandhu,826,knight st,NaN,oswego,IL,60607,"[3541, 751067]"
14654637,3541_751068,Reference person,1991-04-06,29.065734,Female,White,Melissa,M,Norman,824,walnut street,NaN,white plains,NY,13206,"[3541, 751068]"
14654638,3541_751069,Reference person,1983-02-08,37.220983,Male,Latino,Justin,B,Jones,417,logan street,NaN,riverdale,UT,84119,"[3541, 751069]"


In [13]:
getsizeof(df_census) / 1e6

7290.787512

# See if we can reduce memory usage for some columns by converting dtypes

Yes, converting to pandas `Categorical` saves a lot of memory for most of the columns, and also speeds up processing time later.

In [14]:
getsizeof(df_census['relation_to_household_head']) / 1e6

602.603504

In [15]:
getsizeof(df_census['date_of_birth']) / 1e6

562.786366

In [16]:
getsizeof(df_census['middle_initial']) / 1e6

495.252004

In [17]:
getsizeof(df_census['sex']) / 1e6

525.42181

In [18]:
getsizeof(pd.Categorical(df_census['sex'])) / 1e6

7.504066

In [19]:
getsizeof(pd.Categorical(df_census['middle_initial'])) / 1e6

7.506414

In [20]:
getsizeof(df_census['simulant_id']) / 1e6

569.180064

In [21]:
getsizeof(df_census['zipcode']) / 1e6

120.061104

In [22]:
df_census['zipcode'].nunique()

31605

In [23]:
getsizeof(pd.Categorical(df_census['zipcode'])) / 1e6

16.3173

In [24]:
getsizeof(df_census['first_name']) / 1e6

532.262691

In [25]:
getsizeof(pd.Categorical(df_census['first_name'])) / 1e6

16.787843

In [26]:
getsizeof(df_census['last_name']) / 1e6

537.413881

In [27]:
getsizeof(pd.Categorical(df_census['last_name'])) / 1e6

53.77337

In [28]:
getsizeof(df_census['street_name']) / 1e6

578.807787

In [29]:
getsizeof(pd.Categorical(df_census['street_name'])) / 1e6

121.103117

In [30]:
getsizeof(df_census['date_of_birth']) / 1e6

562.786366

In [31]:
getsizeof(pd.Categorical(df_census['date_of_birth'])) / 1e6

33.472907

In [32]:
df_census_raw.columns

Index(['simulant_id', 'first_name_id', 'middle_name_id', 'last_name_id', 'age',
       'sex', 'race_ethnicity', 'date_of_birth', 'address_id', 'state_id',
       'po_box', 'puma', 'guardian_1', 'guardian_2', 'guardian_1_address_id',
       'guardian_2_address_id', 'relation_to_household_head', 'census_year',
       'housing_type', 'random_seed', 'year_of_birth', 'state', 'first_name',
       'middle_initial', 'last_name', 'zipcode', 'street_number',
       'street_name', 'unit_number', 'city', 'mailing_address_street_number',
       'mailing_address_street_name', 'mailing_address_unit_number',
       'mailing_address_po_box', 'mailing_address_state',
       'mailing_address_city', 'mailing_address_zipcode'],
      dtype='object')

In [33]:
df_census_raw.dtypes.value_counts()

object     23
int64      11
float64     3
dtype: int64

In [34]:
# I think all of these should be categorical except for the id's
df_census_raw.dtypes[df_census_raw.dtypes=='object']

simulant_id                      object
first_name_id                    object
middle_name_id                   object
last_name_id                     object
sex                              object
race_ethnicity                   object
date_of_birth                    object
address_id                       object
relation_to_household_head       object
housing_type                     object
state                            object
first_name                       object
middle_initial                   object
last_name                        object
street_number                    object
street_name                      object
unit_number                      object
city                             object
mailing_address_street_number    object
mailing_address_street_name      object
mailing_address_unit_number      object
mailing_address_state            object
mailing_address_city             object
dtype: object

In [35]:
df_census_raw.dtypes.filter(like='_id')

simulant_id               object
first_name_id             object
middle_name_id            object
last_name_id              object
address_id                object
state_id                   int64
guardian_1_address_id    float64
guardian_2_address_id    float64
dtype: object

In [36]:
df_census_raw.date_of_birth.map(type).unique()

array([<class 'str'>], dtype=object)

In [37]:
df_census_raw.dtypes[df_census_raw.dtypes=='int64']

state_id                   int64
po_box                     int64
puma                       int64
guardian_1                 int64
guardian_2                 int64
census_year                int64
random_seed                int64
year_of_birth              int64
zipcode                    int64
mailing_address_po_box     int64
mailing_address_zipcode    int64
dtype: object

In [38]:
df_census_raw.po_box

0              0
1              0
2              0
3              0
4              0
            ... 
15448912    4657
15448913    4657
15448914       0
15448915       0
15448916       0
Name: po_box, Length: 15448917, dtype: int64

In [39]:
getsizeof(df_census_raw.po_box) / 1e6

123.59148

In [40]:
cat_po_box = pd.Categorical(df_census_raw.po_box)
print(getsizeof(cat_po_box) / 1e6)
cat_po_box

31.586138


[0, 0, 0, 0, 0, ..., 4657, 4657, 0, 0, 0]
Length: 15448917
Categories (19983, int64): [0, 1, 2, 3, ..., 19997, 19998, 19999, 20000]

# Open smaller ACS data for testing 

In [41]:
%%time
df_acs = pd.read_csv(f'{output_dir}/household_survey_observer_acs.csv.bz2')
df_acs

CPU times: user 1.97 s, sys: 47.8 ms, total: 2.01 s
Wall time: 2.03 s


,simulant_id,first_name_id,middle_name_id,last_name_id,age,sex,race_ethnicity,date_of_birth,address_id,state_id,...,street_name,unit_number,city,mailing_address_street_number,mailing_address_street_name,mailing_address_unit_number,mailing_address_po_box,mailing_address_state,mailing_address_city,mailing_address_zipcode
0,5616_1361,5616_1361,5616_1361,5616_1361,40.286410,Male,White,1979-12-18,5616_557,28,...,canterbury rd,NaN,brandon,1500,canterbury rd,NaN,0,MS,brandon,39574
1,5616_1362,5616_1362,5616_1362,5616_1361,30.996331,Female,White,1989-04-03,5616_557,28,...,canterbury rd,NaN,brandon,1500,canterbury rd,NaN,0,MS,brandon,39574
2,5616_1363,5616_1363,5616_1363,5616_1361,3.827596,Male,White,2016-06-03,5616_557,28,...,canterbury rd,NaN,brandon,1500,canterbury rd,NaN,0,MS,brandon,39574
3,5616_57184,5616_57184,5616_57184,5616_57184,42.542830,Male,White,1977-09-15,5616_22967,41,...,franklin street,NaN,portland,6328,franklin street,NaN,0,OR,portland,97232
4,5616_57185,5616_57185,5616_57185,5616_57185,34.179504,Female,White,1986-01-26,5616_22967,41,...,franklin street,NaN,portland,6328,franklin street,NaN,0,OR,portland,97232
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153671,3541_848990,3541_848990,3541_848990,3541_524405,2.624840,Female,Latino,2027-09-30,3541_210891,6,...,pantano pl,NaN,julian,2443,pantano pl,NaN,0,CA,julian,95348
153672,3541_855170,3541_855170,3541_855170,3541_594502,2.175033,Female,Black,2028-03-12,3541_391189,42,...,northfield rd,NaN,chambersburg,4111,northfield rd,NaN,0,PA,chambersburg,19041
153673,3541_860702,3541_860702,3541_860702,3541_16126,27.204676,Female,Asian,2003-03-01,3541_508326,12,...,scott church rd,u 28,the villages,1295,scott church rd,u 28,0,FL,the villages,32222
153674,3541_868228,3541_868228,3541_868228,3541_210750,1.219996,Female,White,2029-02-24,3541_313925,27,...,waterfield ln,NaN,scandia,1827,waterfield ln,NaN,0,MN,scandia,56601


In [42]:
df_acs.dtypes

simulant_id                       object
first_name_id                     object
middle_name_id                    object
last_name_id                      object
age                              float64
sex                               object
race_ethnicity                    object
date_of_birth                     object
address_id                        object
state_id                           int64
po_box                             int64
puma                               int64
guardian_1                         int64
guardian_2                         int64
guardian_1_address_id            float64
guardian_2_address_id            float64
survey_date                       object
household_id                       int64
housing_type                      object
random_seed                        int64
year_of_birth                      int64
state                             object
first_name                        object
middle_initial                    object
last_name       

In [43]:
getsizeof(df_acs) / 1e6 # 235.514915 MB for default dtypes

235.514915

In [44]:
getsizeof(df_acs['zipcode']) / 1e6

1.229552

# Try reading ACS data using the `dtype` parameter of `.read_csv()` to load some columns as `Categorical`

In [45]:
%%time
categorical_cols = [
    'sex', 'race_ethnicity', 'middle_initial', 'zipcode', 'state',
    'fake_column', # Does passing a nonexistent column raise an error? No. Great!
]
dtypes = {col: 'category' for col in categorical_cols}
df_acs_cat = pd.read_csv(f'{output_dir}/household_survey_observer_acs.csv.bz2', dtype=dtypes)
df_acs_cat

CPU times: user 2.02 s, sys: 31.9 ms, total: 2.05 s
Wall time: 2.05 s


,simulant_id,first_name_id,middle_name_id,last_name_id,age,sex,race_ethnicity,date_of_birth,address_id,state_id,...,street_name,unit_number,city,mailing_address_street_number,mailing_address_street_name,mailing_address_unit_number,mailing_address_po_box,mailing_address_state,mailing_address_city,mailing_address_zipcode
0,5616_1361,5616_1361,5616_1361,5616_1361,40.286410,Male,White,1979-12-18,5616_557,28,...,canterbury rd,NaN,brandon,1500,canterbury rd,NaN,0,MS,brandon,39574
1,5616_1362,5616_1362,5616_1362,5616_1361,30.996331,Female,White,1989-04-03,5616_557,28,...,canterbury rd,NaN,brandon,1500,canterbury rd,NaN,0,MS,brandon,39574
2,5616_1363,5616_1363,5616_1363,5616_1361,3.827596,Male,White,2016-06-03,5616_557,28,...,canterbury rd,NaN,brandon,1500,canterbury rd,NaN,0,MS,brandon,39574
3,5616_57184,5616_57184,5616_57184,5616_57184,42.542830,Male,White,1977-09-15,5616_22967,41,...,franklin street,NaN,portland,6328,franklin street,NaN,0,OR,portland,97232
4,5616_57185,5616_57185,5616_57185,5616_57185,34.179504,Female,White,1986-01-26,5616_22967,41,...,franklin street,NaN,portland,6328,franklin street,NaN,0,OR,portland,97232
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153671,3541_848990,3541_848990,3541_848990,3541_524405,2.624840,Female,Latino,2027-09-30,3541_210891,6,...,pantano pl,NaN,julian,2443,pantano pl,NaN,0,CA,julian,95348
153672,3541_855170,3541_855170,3541_855170,3541_594502,2.175033,Female,Black,2028-03-12,3541_391189,42,...,northfield rd,NaN,chambersburg,4111,northfield rd,NaN,0,PA,chambersburg,19041
153673,3541_860702,3541_860702,3541_860702,3541_16126,27.204676,Female,Asian,2003-03-01,3541_508326,12,...,scott church rd,u 28,the villages,1295,scott church rd,u 28,0,FL,the villages,32222
153674,3541_868228,3541_868228,3541_868228,3541_210750,1.219996,Female,White,2029-02-24,3541_313925,27,...,waterfield ln,NaN,scandia,1827,waterfield ln,NaN,0,MN,scandia,56601


In [46]:
df_acs_cat.dtypes

simulant_id                        object
first_name_id                      object
middle_name_id                     object
last_name_id                       object
age                               float64
sex                              category
race_ethnicity                   category
date_of_birth                      object
address_id                         object
state_id                            int64
po_box                              int64
puma                                int64
guardian_1                          int64
guardian_2                          int64
guardian_1_address_id             float64
guardian_2_address_id             float64
survey_date                        object
household_id                        int64
housing_type                       object
random_seed                         int64
year_of_birth                       int64
state                            category
first_name                         object
middle_initial                   c

In [47]:
getsizeof(df_acs_cat) / 1e6 # 199.632156 MB after converting some columns to categories

199.632156

# Test whether I can use the `Series.str` accessor on a Categorical

Yes, I can! And it's faster than using it on the original string column!

In [48]:
df_acs_cat['zipcode']

0         39574
1         39574
2         39574
3         97232
4         97232
          ...  
153671    95348
153672    19041
153673    32222
153674    56601
153675    27610
Name: zipcode, Length: 153676, dtype: category
Categories (16612, object): ['10002', '10010', '10011', '10013', ..., '95324', '96142', '97759', '98443']

In [49]:
df_acs_cat['zipcode'].str.startswith('3')

0          True
1          True
2          True
3         False
4         False
          ...  
153671    False
153672    False
153673     True
153674    False
153675    False
Name: zipcode, Length: 153676, dtype: bool

In [50]:
getsizeof(df_acs_cat['zipcode']) / 1e6

1.864451

In [51]:
acs_zip_str = df_acs['zipcode'].astype(str)
%timeit acs_zip_str.str.startswith('3')
%timeit df_acs_cat['zipcode'].str.startswith('3')

28.9 ms ± 129 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
3.96 ms ± 16.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# Write functions to convert between string IDs and integer IDs

In [52]:
df_acs.simulant_id

0           5616_1361
1           5616_1362
2           5616_1363
3          5616_57184
4          5616_57185
             ...     
153671    3541_848990
153672    3541_855170
153673    3541_860702
153674    3541_868228
153675    3541_876077
Name: simulant_id, Length: 153676, dtype: object

In [53]:
2**64 / 1e10 # Padding to width 10 is still very far from overflow for 64-bit integers

1844674407.3709552

In [66]:
%%time
ID_PAD_WIDTH = 10 # must be larger than maximum number of digits in an ID

def id_str_to_int(id_col_str):
#     pattern = r"(?P<seed_id>\d+)_(?P<sim_id>\d+)"
#     id_pieces_df = id_col_str.str.extract(pattern).astype(int)
#     id_col_int = id_pieces_df['seed_id'] * 10**ID_PAD_WIDTH + id_pieces_df['sim_id']
    id_pieces = id_col_str.str.split('_')
    seed_id, sim_id = id_pieces.str[0].astype(int), id_pieces.str[1].astype(int)
    id_col_int = seed_id * 10**ID_PAD_WIDTH + sim_id
    return id_col_int

def id_int_to_str(id_col_int):
    seed_id, sim_id = id_col_int.divmod(10**ID_PAD_WIDTH)
    id_col_str = seed_id.astype(str) + '_' + sim_id.astype(str)
    return id_col_str

acs_sim_id_ints = id_str_to_int(df_acs.simulant_id)
acs_sim_id_ints

CPU times: user 191 ms, sys: 12 ms, total: 203 ms
Wall time: 202 ms


0         56160000001361
1         56160000001362
2         56160000001363
3         56160000057184
4         56160000057185
               ...      
153671    35410000848990
153672    35410000855170
153673    35410000860702
153674    35410000868228
153675    35410000876077
Name: simulant_id, Length: 153676, dtype: int64

In [55]:
getsizeof(acs_sim_id_ints) / 1e6

1.229552

In [56]:
getsizeof(df_acs.simulant_id) / 1e6

10.428819

In [57]:
np.ceil(np.log10(1234567))

7.0

In [58]:
56160000001361 // 10**10

5616

In [59]:
acs_sim_id_ints.divmod(10**10)

(0         5616
 1         5616
 2         5616
 3         5616
 4         5616
           ... 
 153671    3541
 153672    3541
 153673    3541
 153674    3541
 153675    3541
 Name: simulant_id, Length: 153676, dtype: int64,
 0           1361
 1           1362
 2           1363
 3          57184
 4          57185
            ...  
 153671    848990
 153672    855170
 153673    860702
 153674    868228
 153675    876077
 Name: simulant_id, Length: 153676, dtype: int64)

In [60]:
id_int_to_str(acs_sim_id_ints)

0           5616_1361
1           5616_1362
2           5616_1363
3          5616_57184
4          5616_57185
             ...     
153671    3541_848990
153672    3541_855170
153673    3541_860702
153674    3541_868228
153675    3541_876077
Name: simulant_id, Length: 153676, dtype: object

In [61]:
df_acs.simulant_id.str.split('_').str[0]

0         5616
1         5616
2         5616
3         5616
4         5616
          ... 
153671    3541
153672    3541
153673    3541
153674    3541
153675    3541
Name: simulant_id, Length: 153676, dtype: object

# See whether converting the string ID to an integer is faster using `Series.str.extract()` or `Series.str.split()`

It looks like `Series.str.split()` is slightly faster than using a regular expression with `str.extract()`.

In [68]:
def id_str_to_int_regex(id_col_str):
    pattern = r"(?P<seed_id>\d+)_(?P<sim_id>\d+)"
    id_pieces_df = id_col_str.str.extract(pattern).astype(int)
    id_col_int = id_pieces_df['seed_id'] * 10**ID_PAD_WIDTH + id_pieces_df['sim_id']
    return id_col_int

def id_str_to_int_split(id_col_str):
    id_pieces = id_col_str.str.split('_')
    seed_id, sim_id = id_pieces.str[0].astype(int), id_pieces.str[1].astype(int)
    id_col_int = seed_id * 10**ID_PAD_WIDTH + sim_id
    return id_col_int

%timeit id_str_to_int_regex(df_census.simulant_id)
%timeit id_str_to_int_split(df_census.simulant_id)

8.49 s ± 224 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
8.24 s ± 74.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [83]:
%%time
census_sim_id_ints = id_str_to_int_split(df_census.simulant_id)
census_sim_id_ints

CPU times: user 17.5 s, sys: 1.12 s, total: 18.7 s
Wall time: 18.6 s


0           32540000000000
1           32540000000001
2           32540000000002
3           32540000000003
4           32540000000004
                 ...      
14654635    35410000751066
14654636    35410000751067
14654637    35410000751068
14654638    35410000751069
14654639    35410000751070
Name: simulant_id, Length: 7503818, dtype: int64

# See how long it takes to convert IDs from int back to string

It's a bit faster than going the other way. It looks like most of the runtime is due to the conversion of ints to strings, not arithmetic done by `divmod` to extract the correct sub-IDs from the concatenated version.

In [84]:
%timeit id_int_to_str(census_sim_id_ints)

5.42 s ± 32 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [85]:
%timeit census_sim_id_ints.divmod(10**ID_PAD_WIDTH)

62.7 ms ± 196 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


# Does `Series.str[0]` work on any Series of lists of strings, even if it didn't come from `Series.str.split()`?

Yes. Woo hoo!

# What about a Series of lists of ints or other types?

Apparently, yes. That's counterintuitive, but woo hoo!

In [90]:
split = df_acs.simulant_id.str.split('_')
split

0           [5616, 1361]
1           [5616, 1362]
2           [5616, 1363]
3          [5616, 57184]
4          [5616, 57185]
               ...      
153671    [3541, 848990]
153672    [3541, 855170]
153673    [3541, 860702]
153674    [3541, 868228]
153675    [3541, 876077]
Name: simulant_id, Length: 153676, dtype: object

In [91]:
split.map(type)

0         <class 'list'>
1         <class 'list'>
2         <class 'list'>
3         <class 'list'>
4         <class 'list'>
               ...      
153671    <class 'list'>
153672    <class 'list'>
153673    <class 'list'>
153674    <class 'list'>
153675    <class 'list'>
Name: simulant_id, Length: 153676, dtype: object

In [93]:
split.map(lambda lst: (type(lst[0]), type(lst[1])))

0         (<class 'str'>, <class 'str'>)
1         (<class 'str'>, <class 'str'>)
2         (<class 'str'>, <class 'str'>)
3         (<class 'str'>, <class 'str'>)
4         (<class 'str'>, <class 'str'>)
                       ...              
153671    (<class 'str'>, <class 'str'>)
153672    (<class 'str'>, <class 'str'>)
153673    (<class 'str'>, <class 'str'>)
153674    (<class 'str'>, <class 'str'>)
153675    (<class 'str'>, <class 'str'>)
Name: simulant_id, Length: 153676, dtype: object

In [89]:
list(map(list, zip('abced', 'fghij')))

[['a', 'f'], ['b', 'g'], ['c', 'h'], ['e', 'i'], ['d', 'j']]

In [98]:
s = pd.Series(map(list, zip('abced', 'fghij')))
s

0    [a, f]
1    [b, g]
2    [c, h]
3    [e, i]
4    [d, j]
dtype: object

In [96]:
s.str[0], s.str[1]

(0    a
 1    b
 2    c
 3    e
 4    d
 dtype: object,
 0    f
 1    g
 2    h
 3    i
 4    j
 dtype: object)

In [100]:
st = pd.Series((zip('abced', 'fghij')))
st

0    (a, f)
1    (b, g)
2    (c, h)
3    (e, i)
4    (d, j)
dtype: object

In [101]:
st.str[0]

0    a
1    b
2    c
3    e
4    d
dtype: object

In [102]:
si = pd.Series(zip([1,2,3,4], [5,6,7,8]))
si

0    (1, 5)
1    (2, 6)
2    (3, 7)
3    (4, 8)
dtype: object

In [108]:
si.str[0], si.str[1]

(0    1
 1    2
 2    3
 3    4
 dtype: int64,
 0    5
 1    6
 2    7
 3    8
 dtype: int64)

## Oh wait, there's an `expand` option for `Series.str.split()`. Duh, of course there is

But apparently it's a bit slower than either of the other two options. That's silly.

In [104]:
df_acs.simulant_id.str.split('_', expand=True)

,0,1
0,5616,1361
1,5616,1362
2,5616,1363
3,5616,57184
4,5616,57185
...,...,...
153671,3541,848990
153672,3541,855170
153673,3541,860702
153674,3541,868228


In [106]:
def id_str_to_int_split_expand(id_col_str):
    id_pieces = id_col_str.str.split('_', expand=True).astype(int)
    seed_id, sim_id = id_pieces[0], id_pieces[1]
    id_col_int = seed_id * 10**ID_PAD_WIDTH + sim_id
    return id_col_int

%timeit id_str_to_int_split_expand(df_census.simulant_id)

8.77 s ± 113 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Time all three str -> int conversion methods again since I got differing times for the same operation above

Consistent with what I got above, it looks like the runtimes are in the following order:

```
id_str_to_int_split_expand > id_str_to_int_regex > id_str_to_int_split
```

So for some reason, expanding to a dataframe using `str.split(expand=True)` is the slowest, whereas using `str.split(expand=False)` to create a Series of lists is the fastest, and using a regular expression with `str.extract()` is in between.

In [107]:
%timeit id_str_to_int_regex(df_census.simulant_id)
%timeit id_str_to_int_split(df_census.simulant_id)
%timeit id_str_to_int_split_expand(df_census.simulant_id)

9.21 s ± 110 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
8.44 s ± 40.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
9.54 s ± 152 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Check which version of pandas I have installed

`pandas                    1.5.1                    pypi_0    pypi`

In [109]:
!conda list

# packages in environment at /ihme/homes/ndbs/miniconda3/envs/prlpy38:
#
# Name                    Version                   Build  Channel
_libgcc_mutex             0.1                 conda_forge    conda-forge
_openmp_mutex             4.5                       2_gnu    conda-forge
altair                    4.2.0                    pypi_0    pypi
anyio                     3.6.2                    pypi_0    pypi
argon2-cffi               21.3.0                   pypi_0    pypi
argon2-cffi-bindings      21.2.0                   pypi_0    pypi
asttokens                 2.0.8                    pypi_0    pypi
attrs                     22.1.0                   pypi_0    pypi
babel                     2.10.3                   pypi_0    pypi
backcall                  0.2.0                    pypi_0    pypi
beautifulsoup4            4.11.1                   pypi_0    pypi
bleach                    5.0.1                    pypi_0    pypi
blosc                     1.21.1               h83bc5f